## Libraries

In [1]:
import pandas as pd
import numpy as np

## Load features

In [2]:
train = pd.read_pickle('../features/train.pkl')

In [3]:
test = pd.read_pickle('../features/test.pkl')

## Feature selection

In [4]:
from sklearn.feature_selection import VarianceThreshold
from keras.preprocessing import sequence, text

try:
    from sklearn.model_selection import train_test_split
except ImportError:
    from sklearn.cross_validation import train_test_split

Using TensorFlow backend.


In [5]:
features = ['cosin_sim', 'word_share', 'q1_char_num', 'q1_word_num', 'q2_char_num', 'q2_word_num',
            'start_with_same_world', 'rfidf_share', 'char_difference', 'word_difference',
           'seq_simhash_distance', 'shingle_simhash_distance', 'avg_word_len_q1', 'avg_word_len_q2',
           'avg_word_difference', 'unigrams_common_count', 'bigrams_common_count', 'unigrams_common_ratio',
           'bigrams_common_ratio', 'word2vec_q1_mean', 'word2vec_q2_mean']

target = 'is_duplicate'

X = train[features]
y = train[target]

X_test = test[features]

In [7]:
tokenizer = text.Tokenizer(num_words=10000)

max_len = 40
tokenizer.fit_on_texts(list(train.question1.values.astype(str)) + list(train.question2.values.astype(str)))

x1 = tokenizer.texts_to_sequences(train.question1.values.astype(str))
x1 = sequence.pad_sequences(x1, maxlen=max_len)

x2 = tokenizer.texts_to_sequences(train.question2.values.astype(str))
x2 = sequence.pad_sequences(x2, maxlen=max_len)

word_index = tokenizer.word_index

## Oversampling

In [ ]:
pos_train = X[y == 1]
neg_train = X[y == 0]

# Now we oversample the negative class
# There is likely a much more elegant way to do this...
p = 0.165
scale = ((len(pos_train) / (len(pos_train) + len(neg_train))) / p) - 1
while scale > 1:
    neg_train = pd.concat([neg_train, neg_train])
    scale -=1
neg_train = pd.concat([neg_train, neg_train[:int(scale * len(neg_train))]])
print(len(pos_train) / (len(pos_train) + len(neg_train)))

X = pd.concat([pos_train, neg_train])
y = (np.zeros(len(pos_train)) + 1).tolist() + np.zeros(len(neg_train)).tolist()

del pos_train, neg_train

## Normalization

In [8]:
from sklearn.preprocessing import StandardScaler

In [9]:
scaler = StandardScaler()

X = scaler.fit_transform(X)
X_test = scaler.transform(X_test)

## Model

In [10]:
from sklearn.metrics import log_loss

import tensorflow as tf

from keras.layers.advanced_activations import PReLU
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU

#from keras.layers.merge import Concatenate
from keras.layers import Merge

from keras.models import Sequential
from keras.layers import Activation, Dropout, Dense, BatchNormalization
from keras_tqdm import TQDMNotebookCallback

import keras
from keras import backend as K
K.set_image_dim_ordering('tf')

In [33]:
model_q1 = Sequential()
model_q1.add(Embedding(len(word_index) + 1, 32, input_length=max_len, input_shape=(x1.shape[1],)))
model_q1.add(GRU(32, recurrent_dropout=0.2, dropout=0.2, return_sequences=True))
model_q1.add(GRU(64, recurrent_dropout=0.2, dropout=0.2))

model_q2 = Sequential()
model_q2.add(Embedding(len(word_index) + 1, 32, input_length=max_len, input_shape=(x1.shape[1],)))
model_q2.add(GRU(32, recurrent_dropout=0.2, dropout=0.2, return_sequences=True,))
model_q2.add(GRU(64, recurrent_dropout=0.2, dropout=0.2))

model_features = Sequential()
model_features.add(Dense(64, input_shape=(X.shape[1],), activation='relu'))
model_features.add(BatchNormalization())

model_features.add(Dense(128, activation='relu'))
model_features.add(Dropout(0.2))
model_features.add(BatchNormalization())

model_features.add(Dense(256, activation='relu'))
model_features.add(Dropout(0.2))
model_features.add(BatchNormalization())

In [34]:
merged_model = Sequential()
merged_model.add(Merge([model_q1, model_q2, model_features], mode = 'concat'))
#merged_model.add(Concatenate([model_q1, model_q2, model_features]))
merged_model.add(BatchNormalization())

merged_model.add(Dense(16, activation='relu'))
merged_model.add(BatchNormalization())

merged_model.add(Dense(32, activation='relu'))
merged_model.add(Dropout(0.2))
merged_model.add(BatchNormalization())

merged_model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:2: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  from ipykernel import kernelapp as app


In [35]:
merged_model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [36]:
keras_logger = keras.callbacks.TensorBoard(log_dir='../notebooks/tensor_logs/GRU_FC_merge',
                                           histogram_freq=1, write_graph=True, write_images=True)

In [39]:
merged_model.fit([x1, x2, X],
          y,
          batch_size=16 * 16,
          epochs=10,
          verbose=0,
          validation_split=0.1,
          callbacks=[TQDMNotebookCallback(), keras_logger])
          #callbacks=[TQDMNotebookCallback()])

InvalidArgumentError: Tensor must be 4-D with last dim 1, 3, or 4, not [1,1]
	 [[Node: dense_22/bias_0_1 = ImageSummary[T=DT_FLOAT, bad_color=Tensor<type: uint8 shape: [4] values: 255 0 0...>, max_images=3, _device="/job:localhost/replica:0/task:0/cpu:0"](dense_22/bias_0_1/tag, ExpandDims_50/_1067)]]

Caused by op 'dense_22/bias_0_1', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python3.5/dist-packages/zmq/eventloop/ioloop.py", line 177, in start

In [ ]:
#model.save_weights('relu_6w_2o.h5')
model.save('relu_shallow_model.h5')

In [ ]:
model.load_weights('./relu_shallow.h5')

In [ ]:
scores = model.evaluate(X, y, verbose=0, batch_size=4096 * 8)

print("Model validation accuracy: %.2f" % (scores[1]*100))
print("Model validation loss: %.4f" % (scores[0]))

## Generate submission

In [ ]:
preds = model.predict(X_test, batch_size=4096 * 16)
print(preds.shape)

In [ ]:
np.savetxt(
    '../submissions/submission.csv', np.c_[range(len(preds)), preds],
    delimiter=',', header='test_id,is_duplicate', comments='', fmt='%d,%f'
)